In [ ]:
import sys
sys.path.append('../wm_attack/')
sys.path.append('../watermark/')
sys.path.append('../attacks/')
sys.path.append('../models/')
from atk_utils import *
from dct_wm import *
from fgsm import *
from ifgsm import *
from utils import *

In [ ]:
from torchvision import transforms
import torch

In [ ]:
block_size = 8
alpha = 0.1
beta = 20/255
beta_max = 20/255
N = 20
l1 = 0.01
l2 = 0.1
s_a = 0.0005
s_b = 0.0001
steps = 10
eps = 30/255

In [ ]:
model = get_model('resnet50')
T = transforms.ToTensor()
img = cv2.imread('../img/beagle.jpg')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = addborder(T(img),block_size).cuda()
wm = cv2.imread('../img/logo_big.png')
wm = cv2.cvtColor(wm,cv2.COLOR_BGR2RGB)
wm = transforms.Resize(img.size()[-2:])(T(wm)).cuda()

In [ ]:
label = torch.tensor([model(img.unsqueeze(0)).argmax()]).cuda()  # type: ignore
img_wm_perturb = ifgsm_pipeline(img,label,wm,model,alpha,beta,block_size,steps,eps)  # type: ignore
img_wm_perturb_optimize, wm_extra2, a_res, b_res = ifgsm_wm_opti(img,label,wm,model,alpha,beta,block_size,N,l1,l2,s_a,s_b,beta_max,steps,eps) # type: ignore

In [ ]:
img_wm = embed_wm(img,wm,alpha,block_size)
wm_extract = extract_wm(img,img_wm,alpha,block_size)

In [ ]:
wm_extra1 = extract_wm(img,img_wm_perturb,alpha,block_size)

In [ ]:
pltshow(wm_extra1)

In [ ]:
pltshow(wm_extra2)

In [ ]:
pltshow(wm_extract)

In [ ]:
pltshow(img_wm_perturb)

In [ ]:
pltshow(img_wm_perturb_optimize)